In [ ]:
# Задача. Сделать пайплайн обработки файла cars.csv.

# Необходимо посчитать по каждому производителю (поле manufacturer_name):

# кол-во объявлений
# средний год выпуска автомобилей
# минимальную цену
# максимальную цену
# Выгрузить результат в output.csv.

In [13]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as t

def main():
    spark = SparkSession.builder.getOrCreate()
    # тут читаем данные
    df = spark.read.format("csv").option("header", "true").load("cars.csv")
    # df.show(5) # проверка что все читает

    output = (
        df
        .groupBy("manufacturer_name")
        .agg(
            F.count("manufacturer_name").alias("count_ads"),
            F.round(F.avg("year_produced")).cast(t.IntegerType()).alias("avg_year_produced"),
            F.min(F.col("price_usd").cast(t.FloatType())).alias("min_price"),
             F.min(F.col("price_usd").cast(t.FloatType()))
        )       
    )
    # output.show(6) #проверка что происходит агригации
    # тут записыываем в файлик
    df.coalesce(4).write.mode("overwrite").format("json").save("output.json")

main()